# Transfer Learning Practice

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

## Data Steps

### Load Cats vs Dogs Dataset

In [4]:
tfds.disable_progress_bar()
train_ds, validation_ds, test_ds = tfds.load('cats_vs_dogs',
                                             split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"], # 10% for validation, 10% for test
                                             as_supervised=True # Include labels
                                             )

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [5]:
print(type(train_ds)) # See what datasets are

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>


### Pre-process Data

In [7]:
# Make images all 150x150

resize_function = tf.keras.layers.Resizing(150, 150)

train_ds = train_ds.map(lambda x, y: (resize_function(x), y))
validation_ds = validation_ds.map(lambda x, y: (resize_function(x), y))
test_ds = test_ds.map(lambda x, y: (resize_function(x), y))

### Data Augmentation

In [9]:
augmentation_layers = [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
]

def augment_data(data):
  for layer in augmentation_layers:
    data = layer(data)
  return data

In [10]:
train_ds = train_ds.map(lambda x, y: (augment_data(x), y))

### Batch data and use pre-fetching

In [11]:
batch_size = 32

train_ds = train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()
validation_ds = validation_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

## Train Model

### Load Xception Model

In [12]:
base_model = tf.keras.applications.Xception(
    weights = 'imagenet', # Image-net weights are pre-trained
    input_shape = (150, 150, 3),
    include_top=False # Last layer is classification, so exclude it
    )

base_model.trainable = False # Freeze base model

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Create new model

In [13]:
inputs = tf.keras.Input(shape=(150, 150, 3))

# Re-scale input layer required by Xception
scale_layer = tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)

x = scale_layer(inputs)

# Make sure base model is in inference mode
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)  # Regularize output
outputs = tf.keras.layers.Dense(1)(x) # Binary classification (cat or dog)
model = tf.keras.Model(inputs, outputs)

model.summary(show_trainable=True)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer_1 (InputLayer)          │ (None, 150, 150, 3)          │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ rescaling (Rescaling)               │ (None, 150, 150, 3)          │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ xception (Functional)               │ (None, 5, 5, 2048)           │    20,861,480 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ global_average_pooling2d            │ (None, 2048)                 │             0 │    -    │
│ (GlobalAveragePooling2D)            │                              │               │         │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dropout (Dropout)                   │ (None, 2048)                 │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dense (Dense)                       │ (None, 1)                    │         2,049 │    Y    │
└─────────────────────────────────────┴──────────────────────────────┴───────────────┴─────────┘

 Total params: 20,863,529 (79.59 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 20,861,480 (79.58 MB)

### Train new model

In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

epochs = 2
print("Fitting the top layer of the model")
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

Fitting the top layer of the model
Epoch 1/2
291/291 ━━━━━━━━━━━━━━━━━━━━ 104s 313ms/step - binary_accuracy: 0.8740 - loss: 0.2646 - val_binary_accuracy: 0.9703 - val_loss: 0.0813
Epoch 2/2
291/291 ━━━━━━━━━━━━━━━━━━━━ 18s 62ms/step - binary_accuracy: 0.9422 - loss: 0.1319 - val_binary_accuracy: 0.9716 - val_loss: 0.0744


### Fine tune new model

In [15]:
base_model.trainable = True # Batch normalization layers still in inference mode because training = False was passed earlier (so weights don't get messed up)
model.summary(show_trainable=True)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer_1 (InputLayer)          │ (None, 150, 150, 3)          │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ rescaling (Rescaling)               │ (None, 150, 150, 3)          │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ xception (Functional)               │ (None, 5, 5, 2048)           │    20,861,480 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ global_average_pooling2d            │ (None, 2048)                 │             0 │    -    │
│ (GlobalAveragePooling2D)            │                              │               │         │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dropout (Dropout)                   │ (None, 2048)                 │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dense (Dense)                       │ (None, 1)                    │         2,049 │    Y    │
└─────────────────────────────────────┴──────────────────────────────┴───────────────┴─────────┘

 Total params: 20,867,629 (79.60 MB)

 Trainable params: 20,809,001 (79.38 MB)

 Non-trainable params: 54,528 (213.00 KB)

 Optimizer params: 4,100 (16.02 KB)

In [16]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Super low learning rate so Xception weights aren't ruined
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

epochs = 1
print("Fitting the end-to-end model")
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

Fitting the end-to-end model
291/291 ━━━━━━━━━━━━━━━━━━━━ 122s 275ms/step - binary_accuracy: 0.8692 - loss: 0.3394 - val_binary_accuracy: 0.9587 - val_loss: 0.1088


## Evaluate Model

In [17]:
print("Test dataset evaluation")
model.evaluate(test_ds)

Test dataset evaluation
73/73 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - binary_accuracy: 0.9575 - loss: 0.1011


[0.11135406047105789, 0.9552880525588989]

## Save Model

In [18]:
model.save('cat_or_dog.keras')